In [1]:
import pylab as plt
import seaborn as sns
import pandas as pd
import numpy as np
from process_data import process_data

from scipy.stats import randint as sp_randint
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from keras.layers import Dense
from keras.models import Sequential
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,BaggingRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score

2023-02-25 15:26:21.294920: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-25 15:26:22.119153: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /scratch/shyamss/boost_1_76_0/stage/lib:/scratch/shyamss/fftw/lib::/scratch/shyamss/openmpi-4.1.1/install/lib64:/scratch/shyamss/petsc-3.15.1/arch-linux-c-debug/lib:/usr/local/cuda/lib64:/scratch/shyamss/anaconda3/lib
2023-02-25 15:26:22.119276: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_p

In [2]:
# Returns the columnn names and the data
cols, data = process_data("bld1.csv")

/scratch/shyamss/git_repos/CSE23Hackathon/process_data.py:9: DtypeWarning: Columns (10,14,20,22,35,36,37,38,39,40,41,42,43,44,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(filename)


In [3]:
# find indices at which the column names contain 'Zone Air Temperature'
# we want to make these what we're predicting.
# the rest is just inputs to our model
idx_temperature = np.array(['Zone Air Temperature' in col for col in cols])

In [4]:
X = data[cols[~idx_temperature]]
y = data[cols[idx_temperature]]
from sklearn.preprocessing import MinMaxScaler

y1 = y.iloc[:,0:1]
y2 = y.iloc[:,1:2]
y3 = y.iloc[:,2:3]

In [5]:
# Split the data into training and testing sets
X_train, X_test, y1_train, y1_test, y2_train, y2_test, y3_train, y3_test = train_test_split(X, y1, y2, y3, test_size=0.2, random_state = 20) 

MinMax = MinMaxScaler(feature_range= (0,1))
X_train = MinMax.fit_transform(X_train)
X_test = MinMax.transform(X_test)

In [6]:
Acc = pd.DataFrame(index=None, columns=['model','train_Living','test_Living',
                                        'train_Garage','test_Garage',
                                        'train_Attic, test_Attic'])

In [7]:
regressors = [['DecisionTreeRegressor',DecisionTreeRegressor()],
              ['RandomForestRegressor', RandomForestRegressor()]]

In [8]:
for mod in regressors:
    name = mod[0]
    model = mod[1]
    
    model.fit(X_train,y1_train)
    actr1 = r2_score(y1_train, model.predict(X_train))
    acte1 = r2_score(y1_test, model.predict(X_test))
    
    model.fit(X_train,y2_train)
    actr2 = r2_score(y2_train, model.predict(X_train))
    acte2 = r2_score(y2_test, model.predict(X_test))

    model.fit(X_train,y3_train)
    actr3 = r2_score(y3_train, model.predict(X_train))
    acte3 = r2_score(y3_test, model.predict(X_test))
    
    Acc = Acc.append(pd.Series({'model':name, 
                                'train_Living':actr1,'test_Living':acte1,
                                'train_Garage':actr2,'test_Garage':acte2,
                                'train_Attic':actr3,'test_Attic':acte3}),ignore_index=True )

0


/tmp/ipykernel_28508/1728641493.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Acc = Acc.append(pd.Series({'model':name,
/tmp/ipykernel_28508/1728641493.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train,y1_train)


0


/tmp/ipykernel_28508/1728641493.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train,y2_train)
/tmp/ipykernel_28508/1728641493.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train,y3_train)
/tmp/ipykernel_28508/1728641493.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Acc = Acc.append(pd.Series({'model':name,


In [9]:
Acc

,model,train_Living,test_Living,train_Garage,test_Garage,"train_Attic, test_Attic",test_Attic,train_Attic
0,DecisionTreeRegressor,0.999978,0.982486,0.999999,0.995937,NaN,0.987320,0.999999
1,RandomForestRegressor,0.998562,0.991276,0.999774,0.998498,NaN,0.995943,0.999360
